<a href="https://colab.research.google.com/github/LC1332/Luotuo-Text-Embedding/blob/main/notebook/Lyrics_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 歌词分析
这个colab展示了使用Luotuo-Text-Embedding进行歌词分析的demo。

包括了直接使用colab进行分析和使用gradio进行分析两种方式。

In [ ]:
#@title 环境安装和进入
! pip install -U kaleido
! pip install gradio
! pip install transformers
! pip install openTSNE

! git clone https://github.com/LC1332/Luotuo-Text-Embedding

%cd Luotuo-Text-Embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.8 MB/s eta 0:00:

In [ ]:
#@title 导入我们需要的模块
import gradio as gr
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
import torch
from lib.tsne import TSNE_Plot

In [ ]:
#@title 加载模型
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None,
                       pooler_type="cls",
                       temp=0.05,
                       mlp_only_train=False,
                       init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert",
                  trust_remote_code=True,
                  model_args=model_args)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [ ]:
#@title 定义切分句子的函数
def divide_str(s, sep=['\n', '.', '。']):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, 0, mid_len)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos - mid_len):
            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ''
    return s[:best_sep_pos + 1], s[best_sep_pos + 1:]

def strong_divide( s ):
  left, right = divide_str(s)

  if right != '':
    return left, right

  whole_sep = ['\n', '.', '，', '、', ';', ',', '；',\
               '：', '！', '？', '(', ')', '”', '“', \
               '’', '‘', '[', ']', '{', '}', '<', '>', \
               '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
               '·', '…']
  left, right = divide_str(s, sep = whole_sep )

  if right != '':
    return left, right
  
  mid_len = len(s) // 2
  return s[:mid_len], s[mid_len:]

## 直接使用colab

In [ ]:
#@title 定义为给定歌词生成图像的函数
def generate(input_string):
    divided_text = [strong_divide(i) if isinstance(i, str) else strong_divide(i[0]) for i in input_string]
    label = []
    for idx, i in enumerate(input_string):
        if isinstance(i, str):
            label.append('No.{}'.format(idx))
        else:
            label.append(i[1])
    text_left, text_right = [i[0] for i in divided_text], [i[1] for i in divided_text]

    inputs = tokenizer(text_left, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings_left = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
    inputs = tokenizer(text_right, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings_right = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output

    merged_list = text_left + text_right
    merged_embed = torch.cat((embeddings_left, embeddings_right), dim=0)
    print(label)
    tsne_plot = TSNE_Plot(merged_list, merged_embed, label=label * 2, n_annotation_positions=len(merged_list))
    tsne_plot.tsne_plot(n_sentence=len(merged_list))

In [ ]:
#@title 输入万能青年旅店的十首歌的歌词并生成图像进行展示
input_string_wanqing = ['站在能分割世界的桥 还是看不清 在那些时刻 遮蔽我们 黑暗的心 究竟是什么 住在我心里孤独的 孤独的海怪 痛苦之王 开始厌倦 深海的光 停滞的海浪',
         '渔王还想 继续做渔王 而海港已经 不知去向 此刻他醉倒 在洗浴中心 没有潮汐的梦 胸口已暮色苍茫 肥胖的城市 递给他一个 传统的方法 来克制恐慌 卖掉武器 风暴和喉咙 换取饮食 背叛能让你获得自由',
         '溜出时代银行的后门 撕开夜幕和喑哑的平原 越过淡季森林和电 牵引我们黑暗的心 在愿望的最后一个季节 解散清晨还有黄昏 在愿望的最后一个季节 记起我曾身藏利刃 是谁来自山川湖海 却囿于昼夜厨房与爱',
         '傍晚六点下班 换掉药厂的衣裳 妻子在熬粥 我去喝几瓶啤酒 如此生活三十年 直到大厦崩塌 云层深处的黑暗啊 淹没心底的景观 在八角柜台 疯狂的人民商场 用一张假钞 买一把假枪 保卫她的生活 直到大厦崩塌 夜幕覆蓋华北平原 忧伤浸透她的脸',
         '敌视现实 虚构远方 东张西望 一无所长 四体不勤 五谷不分 文不能测字 武不能防身 喜欢养狗 不爱洗头 不事劳作 一无所获 厌恶争执 不善言说 终于沦为沉默的帮凶',
         '切断电缆 朝霞晚风 临时收入 临时生活 切断电缆 数字云烟 免费月光 免费惊险 月光熔铁星 铸我神与心 月光熔铁星 铸我神与心 渤海地产 太行水泥 宗教医保 慈善股票 幻觉贸易 阶级电梯 高级魔术 高级发明 凝聚神与心 建此不夜城 凝聚神与心 建此不夜城',
         '自然赠予你 树冠 微风 肩头的暴雨 片刻后生成 平衡 忠诚 不息的身体 捕食饮水 清早眉间白云生 跳跃漫游 晚来拂面渤海风 朝霞化精灵 轻快 明亮 恒温的伴侣 她与你共存 违背 对抗 相同的命运 爱与疼痛 不觉茫茫道路长 生活历险 并肩莽莽原野荒',
         '开采 我的血肉的火光 发动 新世界的前进的泡影 雷鸣 交织爆破成动荡 此生再不归太行 捶打我天然的沉默 切割我卑微与困惑 面貌已生疏 前方模糊 灵魂在山口又回顾',
         '不速之客 一贫如洗 劳动 饮酒 叹息 夜宿河床枕露珠 测量绘图 爆破合围 加固文明幻景 开山拦河建水库 泥沙沉积 运动停息 随后水鸟隐迹 人造湖泊无颜色',
         '听雷声 滚滚 他默默 闭紧嘴唇 停止吟唱暮色与想念 他此刻沉痛而危险 听雷声 滚滚 他渐渐 感到胸闷 乌云阻拦明月涌河湾 他起身独立向荒原',
         ] 

generate(input_string_wanqing)

['No.0', 'No.1', 'No.2', 'No.3', 'No.4', 'No.5', 'No.6', 'No.7', 'No.8', 'No.9']


In [ ]:
#@title 输入新裤子的十首歌的歌词并生成图像进行展示
input_string_xinkuzi = ['每当浪潮来临的时候 你会不会也伤心 在拥挤孤独的房间里 我已经透不过气 在这冰冷无情的城市里 在摩登颓废的派对里 每当吉他噪音又响起 电流穿过我和你',
         '我最爱去的唱片店 昨天是她的最后一天 曾经让我陶醉的碎片 全都散落在街边 我最爱去的书店 她也没撑过这个夏天 回忆文字流淌著怀念 可是已没什么好怀念 可是你曾经的那些梦 都已变得模糊看不见 那些为了理想的战斗 也不过为了钱 可是我最恨的那个人 他始终没死在我面前 还没年轻就变得苍老 这一生无解',
         '勇敢的你 站在这里 脸庞清瘦却骄傲 在这远方 没人陪伴 只有幻想和烦恼 无聊的 渺小的 反对不公平的世界 没能继续的革命 不欢而散的告别 我倒下后 不敢回头 不能再见的朋友 有人堕落 有人疯了 有人随着风去了',
         '停留在空荡的车站里 看着外面的烟雨 陌生音乐又响起 妳等的人他在哪里 旅客们匆匆的离去 下壹站不知要去哪里 冷冷凄风迷住眼睛 妳爱的人他已变心',
         '依偎在安静的车厢里 越过夜幕下的森林 那首时光的恋曲 飘荡在荒芜的青空 我们默默的向前走 我们还是恋人吗 漫长治愈的旅行 风景美的让人迷惑',
         '每次他又被别人欺骗的时候 他只有音乐 每次这个城市脏得让人无法呼吸的时候 只有音乐 每次他爱的人又被别人抢走的时候 他只有音乐 每次他又无家可归的时候 他只有音乐 弹着吉他的少年',
         '天空在摇摆 找不到存在 简单的概念 都已改变 都已改变 一瞬间 一转眼 我怀疑 一切改变 我发现 你已改变 我 不可能 完美的象机器 和我在一起 拥抱着我 拥抱着我',
         '每当我失去勇气 我显得麻木恐惧 我害怕人们走远 我害怕苍老死去 看你我为我而伤心 看朋友自私势利 没有人再需要我 达到他的目的 我计算不出生活有多长距离 也许明天朋友都会离去',
         '那年的夏天 暴雨和闪电 过期的胶片 失落一点点 为什么流泪 为什么怀念 我失眠至少我还有你的照片 还记得你演的文艺片 写在墙上的那一年 不会变',
         '你就是我的明星 我拼命的想你 不顾一切追随你 我拼命的爱你 就算偶像已死去 虽然你不能再歌唱 无法忘记过去的时光 偶然打开收音机 找到失落的回忆'
         ]

generate(input_string_xinkuzi)

In [ ]:
#@title 输入陈奕迅的十首歌的歌词并生成图像进行展示
input_string_chenyixun = ['如果那两个字没有颤抖 我不会发现 我难受 怎么说出口 也不过是分手 如果对于明天没有要求 牵牵手就像旅游 成千上万个门口 总有一个人要先走',
         '我以为要是唱得用心良苦 妳总会对我多点在乎 我以为虽然爱情 已成往事 千言万语说出来可以互相安抚 期待妳感动 真实的我们难相处 写词的让我 唱出你要的幸福 谁曾经感动 分手的关头才懂得 离开排行榜 更铭心刻骨',
         '拦路雨偏似雪花 饮泣的你冻吗 这风褛我给你磨到有襟花 连调了职也不怕 怎么始终牵挂 苦心选中今天想车你回家 原谅我不再送花 伤口应要结疤 花瓣铺满心里坟场才害怕 如若你非我不嫁 彼此终必火化 一生一世等一天需要代价',
         '不要不要假设我知道 一切一切也都是为我而做 为何这么伟大 如此感觉不到 不说一句的爱有多好 只有一次记得实在接触到 骑着单车的我俩 怀紧贴背的拥抱',
         '天空它像什么 爱情就像什么 几朵云在阴天忘了该往哪儿走 思念和寂寞 被吹进了左耳 也许我记不住可是也忘不掉那时候 那种秘密的快乐',
         '逃避分开的孤独 情愿一起不舒服 其实你那占有欲 咬噬我血肉 怕我也有份教育 未能做空虚的枯木 滞留在挤拥的监狱 明白你有控制欲 我为了大局 上了瘾也不戒毒',
         '欢笑声 欢呼声 炒热气氛 心却很冷 聚光灯 是种蒙恩 我却不能 喊等一等 我真佩服我 还能幽默 掉眼泪时 用笑掩过 怕人看破 顾虑好多 不谈寂寞 我们就都快活',
         '仍然没有遇到 那位跟我绝配的恋人 你根本也未有出现 还是已然逝去 怀疑在某一个国度里的某一年 还未带我到世上那天 存在过一位等我爱的某人 夜夜为我失眠 从来未相识 已不在这个人 极其实在却像个虚构角色 莫非今生原定陪我来 却去了错误时代',
         '我来到 你的城市 走过你来时的路 想像著 没我的日子 你是怎样的孤独 拿着你 给的照片 熟悉的那一条街 只是没了你的画面 我们回不到那天',
         '徘徊过多少橱窗 住过多少旅馆 才会觉得分离也并不冤枉 感情是用来浏览 还是用来珍藏 好让日子天天都过得难忘 熬过了多久患难 湿了多少眼眶 才能知道伤感是爱的遗产 流浪几张双人床 换过几次信仰 才让戒指义无反顾的交换',
         ]

generate(input_string_chenyixun)

In [ ]:
#@title （可选）在输入歌词的还可以同时输入每个歌词的歌名一起生成图像展示
input_string_wanqing_with_name = [('站在能分割世界的桥 还是看不清 在那些时刻 遮蔽我们 黑暗的心 究竟是什么 住在我心里孤独的 孤独的海怪 痛苦之王 开始厌倦 深海的光 停滞的海浪', '秦皇岛'),
         ('渔王还想 继续做渔王 而海港已经 不知去向 此刻他醉倒 在洗浴中心 没有潮汐的梦 胸口已暮色苍茫 肥胖的城市 递给他一个 传统的方法 来克制恐慌 卖掉武器 风暴和喉咙 换取饮食 背叛能让你获得自由', '大石碎胸口'),
         ('溜出时代银行的后门 撕开夜幕和喑哑的平原 越过淡季森林和电 牵引我们黑暗的心 在愿望的最后一个季节 解散清晨还有黄昏 在愿望的最后一个季节 记起我曾身藏利刃 是谁来自山川湖海 却囿于昼夜厨房与爱', '揪心的玩笑与漫长的白日梦'),
         ('傍晚六点下班 换掉药厂的衣裳 妻子在熬粥 我去喝几瓶啤酒 如此生活三十年 直到大厦崩塌 云层深处的黑暗啊 淹没心底的景观 在八角柜台 疯狂的人民商场 用一张假钞 买一把假枪 保卫她的生活 直到大厦崩塌 夜幕覆蓋华北平原 忧伤浸透她的脸', '杀死那个石家庄人'),
         '敌视现实 虚构远方 东张西望 一无所长 四体不勤 五谷不分 文不能测字 武不能防身 喜欢养狗 不爱洗头 不事劳作 一无所获 厌恶争执 不善言说 终于沦为沉默的帮凶',
         '切断电缆 朝霞晚风 临时收入 临时生活 切断电缆 数字云烟 免费月光 免费惊险 月光熔铁星 铸我神与心 月光熔铁星 铸我神与心 渤海地产 太行水泥 宗教医保 慈善股票 幻觉贸易 阶级电梯 高级魔术 高级发明 凝聚神与心 建此不夜城 凝聚神与心 建此不夜城',
         ('自然赠予你 树冠 微风 肩头的暴雨 片刻后生成 平衡 忠诚 不息的身体 捕食饮水 清早眉间白云生 跳跃漫游 晚来拂面渤海风 朝霞化精灵 轻快 明亮 恒温的伴侣 她与你共存 违背 对抗 相同的命运 爱与疼痛 不觉茫茫道路长 生活历险 并肩莽莽原野荒', '山雀'),
         '开采 我的血肉的火光 发动 新世界的前进的泡影 雷鸣 交织爆破成动荡 此生再不归太行 捶打我天然的沉默 切割我卑微与困惑 面貌已生疏 前方模糊 灵魂在山口又回顾',
         '不速之客 一贫如洗 劳动 饮酒 叹息 夜宿河床枕露珠 测量绘图 爆破合围 加固文明幻景 开山拦河建水库 泥沙沉积 运动停息 随后水鸟隐迹 人造湖泊无颜色',
         '听雷声 滚滚 他默默 闭紧嘴唇 停止吟唱暮色与想念 他此刻沉痛而危险 听雷声 滚滚 他渐渐 感到胸闷 乌云阻拦明月涌河湾 他起身独立向荒原',
]
generate(input_string_wanqing_with_name)

## Gradio

In [ ]:
#@title 定义为gradio生成图像的函数
def generate_image(text_input):
    # 将输入的文本按行分割并保存到列表中
    text_input = text_input.split('\n')
    label = []
    for idx, i in enumerate(text_input):
        if '#' in i:
            label.append(i[i.find('#') + 1:])
            text_input[idx] = i[:i.find('#')]
        else:
            label.append('No.{}'.format(idx))

    divided_text = [strong_divide(i) for i in text_input]
    text_left, text_right = [i[0] for i in divided_text], [i[1] for i in divided_text]
    inputs = tokenizer(text_left, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
      embeddings_left = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
    inputs = tokenizer(text_right, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
      embeddings_right = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output

    merged_list = text_left + text_right
    merged_embed = torch.cat((embeddings_left, embeddings_right), dim=0)
    tsne_plot = TSNE_Plot(merged_list, merged_embed, label=label * 2, n_annotation_positions=len(merged_list))
    fig = tsne_plot.tsne_plot(n_sentence=len(merged_list), return_fig=True)
    return fig

In [ ]:
#@title 创建 Gradio 界面
with gr.Blocks() as demo:
    name = gr.inputs.Textbox(lines=20,
          placeholder='在此输入歌词，每一行为一个输入，如果需要输入歌词对应的歌名，请用#隔开\n例如：听雷声 滚滚 他默默 闭紧嘴唇 停止吟唱暮色与想念 他此刻沉痛而危险 听雷声 滚滚 他渐渐 感到胸闷 乌云阻拦明月涌河湾 他起身独立向荒原#河北墨麒麟')
    output = gr.Plot()
    btn = gr.Button("Generate")
    btn.click(fn=generate_image, inputs=name, outputs=output, api_name="generate-image")

demo.launch(debug=True)

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning:

Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components

/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning:

`optional` parameter is deprecated, and it has no effect

/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning:

`numeric` parameter is deprecated, and it has no effect



Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>